In [ ]:
model_1.save('2_3_2021_6am.tf')
model_1.save_weights('2_3_2021_6am_weights.tf')


INFO:tensorflow:Assets written to: 2_3_2021_6am.tf/assets


In [ ]:
np.save('img_data',img_data)
np.save('onehot_encoded',onehot_encoded)

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg

import keras as k
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [ ]:
from random import shuffle
#%debug
def create_dataset(img_folder,IMG_HEIGHT=256, IMG_WIDTH=256):
    tic()
    i=1
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        files = os.listdir(os.path.join(img_folder, dir1))
        shuffle(files)
        i=0
        for file in files:
          if file.endswith('.png'):
            if (i%100 == 0):
              print(i)
              #toc()
              #tic()
            #if (i == 2000):
              #break
            i = i+1
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
img_folder=r'/content/drive/MyDrive/CNN_image_3/CNN_image_3/CNN_image_3'
img_data, class_name =create_dataset(img_folder)
toc()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
Elapsed time: 1567.432801 seconds.



In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = class_name
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1001, :])])
print(inverted)

['Over' 'Over' 'Over' ... 'Under' 'Under' 'Under']
[1 1 1 ... 2 2 2]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
['Over']


In [ ]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[999, :])])
print(inverted)

['Over']


In [ ]:
class Same_net(tf.keras.Model):
    def __init__(self,
                     filters_1x1=8,
                     filters_3x3_reduce=8,
                     filters_3x3=8,
                     filters_5x5_reduce=8,
                     filters_5x5=8,
                     filters_pool_proj=8,
                     name='inception_3a',
                     inp_shapes = (256, 256, 3)):
      
        super(Same_net, self).__init__()
        self.conv1x1 = tf.keras.layers.Conv2D(filters_1x1, (1, 1), padding='same')
        
        self.conv3x3_reduce = tf.keras.layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same')
        self.conv3x3 = tf.keras.layers.Conv2D(filters_3x3, (3, 3), padding='same')
        
        self.conv5x5_reduce = tf.keras.layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same')
        self.conv5x5 = tf.keras.layers.Conv2D(filters_5x5, (5, 5), padding='same')
        
        self.convpool = tf.keras.layers.Conv2D(filters_1x1, (1, 1), padding='same')
        
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3),strides=(1, 1), padding='same')
        self.Concat = tf.keras.layers.Concatenate()

    def call(self, inputs):
        
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)
        
        b2 = self.conv3x3_reduce(inputs)
        b2_act = self.act(b2)
        b2_2 = self.conv3x3(b2_act)
        b2_2_act = self.act(b2_2)
        
        b3 = self.conv5x5_reduce(inputs)
        b3_act = self.act(b3)
        b3_2 = self.conv5x5(b3_act)
        b3_2_act = self.act(b3_2)
        
        b4 = self.max_pool(inputs)
        b4_2 = self.convpool(b4)
        b4_2_act = self.act(b4_2)
               
        output = self.Concat([b1_act, b2_2_act, b3_2_act, b4_2_act])
        return output

In [ ]:
class Encod_2(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Encod_2, self).__init__()
        self.conv1x1 = tf.keras.layers.Conv2D(filters_1x1, (3, 3), padding='same')
        self.conv1x1_2 = tf.keras.layers.Conv2D(filters_1x1, (3, 3), padding='same')
        self.max_pool2 = tf.keras.layers.MaxPool2D((4, 4),strides=(3, 3), padding='same')
        self.act = tf.keras.layers.Activation('relu')
    def call(self, inputs):
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)  
        b2 = self.conv1x1_2(b1_act)
        b2_act = self.act(b2)  
        b3 =   self.max_pool2(b2_act) 
        return b3  

In [ ]:
class Encod_comp(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Encod_comp, self).__init__()
        self.b1 = Same_net()
        #self.b12 = Same_net()
        #self.b13 = Same_net()
        
        self.b2 = Encod_2()
        self.b22 = Encod_2()
        self.b23 = Encod_2()

    def call(self, inputs):
        l1 = self.b1(inputs)
        l2 = self.b2(l1)
        #l3 = self.b12(l2)
        l4 = self.b22(l2)
        #l5 = self.b13(l4)
        l6 = self.b23(l4)

               
        return l6

In [ ]:
class Class_out(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Class_out, self).__init__()
        self.Flat = tf.keras.layers.Flatten()
        self.D1 = tf.keras.layers.Dense(150, activation='relu')
        self.D2 = tf.keras.layers.Dense(40, activation='relu')
        self.D3 = tf.keras.layers.Dense(12, activation='relu')
        self.D4 = tf.keras.layers.Dense(4, activation='softmax')
        #self.D5 = tf.keras.layers.Softmax()
        #self.D0 = Encod_comp()

    def call(self, inputs):
      
        #l0 = self.D0(inputs)
        #l1 = self.Flat(l0)

        l1 = self.Flat(inputs)
        l2 = self.D1(l1)
        l3 = self.D2(l2)
        l4 = self.D3(l3)
        l5 = self.D4(l4)
        #l6 = self.D5(l5)
               
        return l5

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,filters_1x1=8,same_pad = 0):
        super(Decoder, self).__init__()
        if same_pad :
          self.conv1x1 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='same')
          self.conv1x1_2 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='same')
        else:
          self.conv1x1 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='valid')
          self.conv1x1_2 = tf.keras.layers.Conv2DTranspose(filters_1x1, (3, 3), padding='valid')
        self.UpSampling2D = tf.keras.layers.UpSampling2D((2, 2), interpolation='bilinear')
        self.act = tf.keras.layers.Activation('sigmoid')

    def call(self, inputs):
        b1 = self.conv1x1(inputs)
        b1_act = self.act(b1)  
        b2 = self.conv1x1_2(b1_act)
        b2_act = self.act(b2)  
        b3 =   self.UpSampling2D(b2_act) 
        return b3  

In [ ]:
class Decod_comp(tf.keras.Model):
    def __init__(self,filters_1x1=8):
        super(Decod_comp, self).__init__()
        self.b1 = Same_net()
        #self.b12 = Same_net()
        #self.b13 = Same_net()
        #self.b14 = Same_net()
        
        self.b2 = Decoder(same_pad = 0)
        self.b22 = Decoder(same_pad = 0)
        self.b23 = Decoder(same_pad = 1)
        self.b24 = Decoder(filters_1x1=3,same_pad = 1)

    def call(self, inputs):
        l1 = self.b1(inputs)
        l2 = self.b2(l1)
        #l3 = self.b12(l2)
        l4 = self.b22(l2)
        #l5 = self.b13(l4)
        l6 = self.b23(l4)
        #l7 = self.b14(l6)
        l8 = self.b24(l6)

               
        return l8

In [ ]:
class Model_comp(tf.keras.Model):
    def __init__(self,filters_1x1=6):
        super(Model_comp, self).__init__()
        self.encoder = Encod_comp()
        self.decoder = Decod_comp()
        self.classifier = Class_out()
    def call(self, inputs):
        l1 = self.encoder(inputs)
        l2 = self.decoder(l1)
        l3 = self.classifier(l1)
        return [l2,l3]

In [ ]:
img_data=np.array(img_data)

In [ ]:
steps_per_epoch = 400
model_1 = Model_comp()
model_1.compile(optimizer='adam', loss=['mean_squared_error' , 'categorical_crossentropy'], loss_weights=[1,1],metrics=['accuracy'])
#model_1.compile(loss="mean_squared_error", optimizer='RMSprop')
history = model_1.fit(x=img_data,
    y=[img_data, onehot_encoded],
    batch_size=64,
    epochs=20,
    verbose=1,
    callbacks=None,
    validation_split=0.2,
    shuffle=True)


Epoch 1/20
191/191 [==============================] - 45s 199ms/step - loss: 1.0796 - output_1_loss: 0.1555 - output_2_loss: 0.9241 - output_1_accuracy: 0.6200 - output_2_accuracy: 0.6093 - val_loss: 1.4078 - val_output_1_loss: 0.0974 - val_output_2_loss: 1.3104 - val_output_1_accuracy: 0.7965 - val_output_2_accuracy: 0.5243
Epoch 2/20
191/191 [==============================] - 37s 195ms/step - loss: 0.3466 - output_1_loss: 0.1103 - output_2_loss: 0.2363 - output_1_accuracy: 0.7663 - output_2_accuracy: 0.9124 - val_loss: 0.9105 - val_output_1_loss: 0.0808 - val_output_2_loss: 0.8297 - val_output_1_accuracy: 0.7965 - val_output_2_accuracy: 0.7230
Epoch 3/20
191/191 [==============================] - 37s 192ms/step - loss: 0.2113 - output_1_loss: 0.0762 - output_2_loss: 0.1351 - output_1_accuracy: 0.7661 - output_2_accuracy: 0.9530 - val_loss: 0.6504 - val_output_1_loss: 0.0683 - val_output_2_loss: 0.5821 - val_output_1_accuracy: 0.8037 - val_output_2_accuracy: 0.8095
Epoch 4/20
191/191 

In [ ]:
model_1.save('2_3_2021_6am.tf')
model_1.save_weights('2_3_2021_6am_weights.tf')


INFO:tensorflow:Assets written to: 2_3_2021_6am.tf/assets


In [ ]:
np.save('img_data',img_data)
np.save('onehot_encoded',onehot_encoded)

In [ ]:
history = model_1.fit(x=img_data,
    y=[img_data, onehot_encoded],
    batch_size=64,
    epochs=200,
    verbose=1,
    callbacks=None,
    validation_split=0.2,
    shuffle=True)

Epoch 1/200
191/191 [==============================] - 37s 191ms/step - loss: 0.0532 - output_1_loss: 0.0447 - output_2_loss: 0.0085 - output_1_accuracy: 0.8462 - output_2_accuracy: 0.9984 - val_loss: 0.1960 - val_output_1_loss: 0.0524 - val_output_2_loss: 0.1436 - val_output_1_accuracy: 0.8527 - val_output_2_accuracy: 0.9744
Epoch 2/200
191/191 [==============================] - 36s 190ms/step - loss: 0.0523 - output_1_loss: 0.0442 - output_2_loss: 0.0080 - output_1_accuracy: 0.8472 - output_2_accuracy: 0.9982 - val_loss: 0.1527 - val_output_1_loss: 0.0520 - val_output_2_loss: 0.1006 - val_output_1_accuracy: 0.8540 - val_output_2_accuracy: 0.9744
Epoch 3/200
191/191 [==============================] - 36s 190ms/step - loss: 0.0506 - output_1_loss: 0.0438 - output_2_loss: 0.0068 - output_1_accuracy: 0.8479 - output_2_accuracy: 0.9985 - val_loss: 0.2023 - val_output_1_loss: 0.0514 - val_output_2_loss: 0.1509 - val_output_1_accuracy: 0.8547 - val_output_2_accuracy: 0.9744
Epoch 4/200
191/

In [ ]:
model_1.save('2_3_2021_7am.tf')
model_1.save_weights('2_3_2021_7am_weights.tf')


INFO:tensorflow:Assets written to: 2_3_2021_7am.tf/assets


In [ ]:
np.save('img_datap',img_data)
np.save('onehot_encodedp',onehot_encoded)